Use this notebook to get Spotify audio data

Useful resources:
- https://stmorse.github.io/journal/spotify-api.html
- https://medium.com/@lorelablaka/extract-data-using-spotify-api-889222835bf4

In [1]:
# dependencies
import requests
import json
import pandas as pd
import re
import timeit
from IPython.display import clear_output

In [197]:
# Spotify API credentials
from config import spotify_client_ID as sp_client, spotify_client_secret as sp_secret

In [198]:
# read in top40 data
top40_df = pd.read_csv("../00_data/top40_1997_2022_raw.csv")

# store columns as lists
week = top40_df['week']
position = top40_df['position']
song = top40_df['song']
artist = top40_df['artist']

# practice list of 10 artists
artist = artist[0:7].tolist() + artist[8:11].tolist()
song = song[0:7].tolist() + song[8:11].tolist()

In [199]:
# get an authorization token
# https://developer.spotify.com/documentation/general/guides/authorization/client-credentials/

auth_token = 'https://accounts.spotify.com/api/token'

# POST token request

auth_response = requests.post(auth_token, {
    'grant_type': 'client_credentials',
    'client_id': sp_client,
    'client_secret': sp_secret,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [200]:
# https://developer.spotify.com/documentation/general/guides/authorization/use-access-token/

# send a GET request to the API server with the access token in the header

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# paste final function here:

In [201]:
# function to get genres and audio features for all top40 tracks
# input: a dataframe of top40 charts with [week, position, song, artist] columns

def get_audio_features(weekly_charts):
    
    # change input dataframe to a dictionary
    top40_dict = weekly_charts.to_dict(orient='index') 
    
    # base URL for API calls
    base_url = 'https://api.spotify.com/v1/'

    # empty lists to store all genres, artist IDs in the dataset
    all_genres = []
    all_artist_genres = []
    artist_ids = []

    for chart_entry in top40_dict.items():

        ################################
        # search artists
        ################################

        # use an 'item' search for artist and track
        # https://developer.spotify.com/documentation/web-api/reference/#/operations/search
        query_artist = re.sub('\s', '%20', chart_entry[1]['artist'])
        query = f'search?q=artist:{query_artist}&type=artist&limit=5'
        artist_search_results = requests.get(base_url + query, headers=headers).json()

        ################################
        # search artists' top tracks
        ################################

        # save all IDs from artist search results
        id_list = []
        #for result in artist_search_results['artists']['items']:
        for result in artist_search_results['artists']['items']:
            artist_id = result['id']
            id_list.append(artist_id)
        chart_entry[1]['id_list'] = id_list
        
        # for each artist ID, search that artist's top tracks
        # https://api.spotify.com/v1/artists/{id}/top-tracks
        hit_list = []
        for artist in id_list:
            
            artist_hits = requests.get(base_url + 'artists/' + artist + '/top-tracks?market=US', 
                                       headers=headers).json()
            hit_list.append(artist_hits)
            chart_entry[1]['hit_list'] = hit_list


        ################################
        # save artist and track URIs
        ################################

        # search each artist's hits for the track of interest
        for track in hit_list[0]['tracks']:
            
            chart_entry[1]['artist_uri'] = 'NA'
            chart_entry[1]['song_uri'] = 'NA'

            # Spotify track and artist info
            track_artist = track['artists'][0]['name']
            track_artist_uri = track['artists'][0]['uri']
            track_title = track['name']
            track_title_uri = track['uri']
            
            # simplify artist/song strings to search chart instances against spotify instances
            compare_track_artist = track_artist.replace(' ', '').casefold()
            compare_track_title = track_title.replace(' ', '').casefold()
            
            compare_chart_artist = chart_entry[1]['artist'].replace(' ', '').casefold()
            compare_chart_song = chart_entry[1]['song'].replace(' ', '').casefold()
            
            # if the artist and song title are found, save the relevant info
            if compare_chart_artist == compare_track_artist and compare_chart_song == compare_track_title:
                
                chart_entry[1]['artist_uri'] = track_artist_uri
                chart_entry[1]['song_uri']  = track_title_uri

                # continue to the next iteration once a match is found and saved
                break
                    
        ##################################
        # get audio features for all songs
        ##################################
        
        # empty lists to hold desired audio feature scores
        acousticness = []
        duration = []
        danceability = []
        energy = []
        instrumentalness = []
        key = []
        liveness = []
        loudness = []
        mode = []
        speechiness = []
        tempo = []
        time_signature = []
        valence = []
    
        # make the API call for track audio features
        # https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features
    
        song_id = re.search('(?:spotify:track:)(\w+)', chart_entry[1]['song_uri']).group(1)
        chart_entry[1]['song_id'] = song_id
        # API call
        audio = requests.get(base_url + 'audio-features/' + song_id, headers=headers).json()
    
        # save audio features to their respective lists
        chart_entry[1]['acousticness'] = audio['acousticness']
        chart_entry[1]['duration'] = audio['duration_ms']
        chart_entry[1]['danceability'] = audio['danceability']
        chart_entry[1]['energy'] = audio['energy']
        chart_entry[1]['instrumentalness'] = audio['instrumentalness']
        chart_entry[1]['key'] = audio['key']
        chart_entry[1]['liveness'] = audio['liveness']
        chart_entry[1]['loudness'] = audio['loudness']
        chart_entry[1]['mode'] = audio['mode']
        chart_entry[1]['speechiness'] = audio['speechiness']
        chart_entry[1]['tempo'] = audio['tempo']
        chart_entry[1]['time_signature'] = audio['time_signature']
        chart_entry[1]['valence'] = audio['valence']
                
                
        ########################
        # get artist genre data
        ########################
        
        # get artist IDs to use in search
        artist_id = re.search('(?:spotify:artist:)(\w+)', chart_entry[1]['artist_uri']).group(1)
        chart_entry[1]['artist_id'] = artist_id
        artist_ids.append(artist_id)

        # save artist-song combinations to use in dataframe construction
        chart_entry[1]['combo'] = artist_id + song_id
        
        # get the artist's genre(s)
        # Spotify 'artist' search
        # https://developer.spotify.com/documentation/web-api/reference/#/operations/get-an-artist
        
#        try:
        # API call
        get_artist = requests.get(base_url + 'artists/' + artist_id, headers=headers).json()
        artist_genres = get_artist['genres']
        # format genre strings
        artist_genres = [x.replace(' ', '_').lower() for x in artist_genres]
        chart_entry[1]['artist_genres'] = artist_genres
#        print(artist_genres)

#        # for artists with 'NA' URIs
#        except:
#            artist_genres = ['no_genre_data']        
        
    
        # empty dictionary to hold all genres associated with each artist
        artist_genre_dict = {}
        artist_genre_dict[chart_entry[1]['artist_id']] = artist_genres
        all_artist_genres.append(artist_genre_dict)
        
        # save new genres to the list of all genres
        for genre in artist_genres:
            if genre not in all_genres:
                all_genres.append(genre)
    
    
###############
# WORKS TO HERE
###############       

    #        # log progress
    #        clear_output()
    #        checkpoint = timeit.default_timer()
    #        print(f'Found genres for {(a+1)}/{len(artist_ids)} artists ({(a+1)/len(artist_ids)*100:.2f}% complete)')
    #        print(f'{(checkpoint-start)/60:.2f} minutes elapsed')
    #
    #    print('Saving data...')

    all_genres_dict = {}

    for genre in all_genres:
        all_genres_dict[genre] = [0] * len(weekly_charts)
    
    # create a dataframe chart data
    chart_df = pd.DataFrame.from_dict(top40_dict, orient='index')

    # create a dataframe with genre data, sort df columns
    genre_df = pd.DataFrame(all_genres_dict)
    genre_df.columns = sorted(genre_df.columns.tolist())
    genre_df['combo'] = chart_df['combo']
    genre_df.set_index('combo', inplace=True)

    # assign 1's to each artist's genre                
    for column in genre_df:
        
        for row, combo in enumerate(genre_df.index):
            
            for genre in chart_df['artist_genres'][row]:
                                    
                if genre == column:
                    genre_df.loc[combo, column] = 1

                
#    #######################################
#    # drop uncommon genres from the dataset
#    #######################################
#
#    # get the sum of each genre column (exclude the last column)
#    genre_counts = genre_df.sum(axis=0)
#    genre_counts_df = pd.DataFrame(genre_counts, columns=['count'])
#
#    # get genres to drop from the dataset
#    # keeping only the top 50 most common genres
#    drop_genres = genre_counts_df.sort_values(['count'],ascending=False)[50:].index
#
#    # remove low-count genres from the dataset
#    genres_df = full_df.drop(columns=drop_genres)


    ################################
    # output all data to a dataframe
    ################################

    # make a copy of the input dataframe
    trimmed_df = chart_df.copy()
    
    # drop unwanted columns
    trimmed_df.drop(columns = ['id_list', 'hit_list', 'artist_genres', 'artist_uri', 'song_uri'], inplace=True)
    
    # reorder columns for aesthetics
    trimmed_df = trimmed_df[['week', 'position', 'artist', 'song', 'artist_id', 'song_id', 'combo',
                           'acousticness', 'danceability', 'duration', 'energy', 'instrumentalness',
                           'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
                           'time_signature', 'valence']]
    
    # merge all data into a final dataframe
    output_df = trimmed_df.merge(genre_df, how="inner", on="combo")
#
#    # log progress
#    checkpoint = timeit.default_timer()
#    print('Data saved')
#    print(f'Total runtime: {(checkpoint-start)/60:.2f} minutes')
#    print('Done!')
#    #print(type(output_df))
#    #print(type(genre_df))
#    #print(type(full_df))

    # return the dataframe
    return output_df

In [202]:
top40_audio = get_audio_features(top40_df.head(10))

AttributeError: 'NoneType' object has no attribute 'group'

In [739]:
artist_hits = requests.get(base_url + 'artists/' + '1saEviAIhddEAjIOvZEeDi' + '/top-tracks?market=US', headers=headers).json()
artist_hits

{'tracks': [{'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2W2MyOwxAxKWGxWVk0vw8v'},
      'href': 'https://api.spotify.com/v1/artists/2W2MyOwxAxKWGxWVk0vw8v',
      'id': '2W2MyOwxAxKWGxWVk0vw8v',
      'name': 'Original Broadway Cast of Moulin Rouge! The Musical',
      'type': 'artist',
      'uri': 'spotify:artist:2W2MyOwxAxKWGxWVk0vw8v'}],
    'external_urls': {'spotify': 'https://open.spotify.com/album/1catLRW8LZfugEmfjnTWCk'},
    'href': 'https://api.spotify.com/v1/albums/1catLRW8LZfugEmfjnTWCk',
    'id': '1catLRW8LZfugEmfjnTWCk',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab67616d0000b2730be5bb9835436e3a5d03820c',
      'width': 640},
     {'height': 300,
      'url': 'https://i.scdn.co/image/ab67616d00001e020be5bb9835436e3a5d03820c',
      'width': 300},
     {'height': 64,
      'url': 'https://i.scdn.co/image/ab67616d000048510be5bb9835436e3a5d03820c',
      'width': 64}],
    'name'

In [393]:
for track in artist_hits['tracks']:
    testd[5]['song']
    song_regex = f'(?:\'name\': \')({testd[5]['song']})'
        
    # extract IDs using regex
    print(re.search(song_regex, str(track)).group(1))


SyntaxError: f-string: unmatched '[' (2365545050.py, line 3)

In [418]:
for i in testd:
    
    if re.search(testd[i]['song'],str(artist_hits['tracks'][4])):
        print(testd[i]['song'])
        song_id = re.search(testd[i]['song'])
        print(song_id)
    else:
        pass

Do You Know


TypeError: string indices must be integers

In [379]:
testd = top40_df.head(10).to_dict(orient='index')

In [203]:
def top40_dict(dfin):
    
    # change input dataframe to a dictionary
    top40_dict = dfin.to_dict(orient='index') 
    
    # base URL for API calls
    base_url = 'https://api.spotify.com/v1/'

    # empty lists to store all genres, artist IDs in the dataset
    all_genres = []
    all_artist_genres = []
    artist_ids = []

    for chart_entry in top40_dict.items():

        ################################
        # search artists
        ################################

        # use an 'item' search for artist and track
        # https://developer.spotify.com/documentation/web-api/reference/#/operations/search
        query_artist = re.sub('\s', '%20', chart_entry[1]['artist'])
        query = f'search?q=artist:{query_artist}&type=artist&limit=5'
        artist_search_results = requests.get(base_url + query, headers=headers).json()

        ################################
        # search artists' top tracks
        ################################

        # save all IDs from artist search results
        id_list = []
        #for result in artist_search_results['artists']['items']:
        for result in artist_search_results['artists']['items']:
            artist_id = result['id']
            id_list.append(artist_id)
        chart_entry[1]['id_list'] = id_list
        
        # for each artist ID, search that artist's top tracks
        # https://api.spotify.com/v1/artists/{id}/top-tracks
        hit_list = []
        for artist in id_list:
            
            artist_hits = requests.get(base_url + 'artists/' + artist + '/top-tracks?market=US', 
                                       headers=headers).json()
            hit_list.append(artist_hits)
            chart_entry[1]['hit_list'] = hit_list


        ################################
        # save artist and track URIs
        ################################

        # search each artist's hits for the track of interest
        for track in hit_list[0]['tracks']:
            
            chart_entry[1]['artist_uri'] = 'NA'
            chart_entry[1]['song_uri'] = 'NA'

            # Spotify track and artist info
            track_artist = track['artists'][0]['name']
            track_artist_uri = track['artists'][0]['uri']
            track_title = track['name']
            track_title_uri = track['uri']
            
            # simplify artist/song strings to search chart instances against spotify instances
            compare_track_artist = track_artist.replace(' ', '').casefold()
            compare_track_title = track_title.replace(' ', '').casefold()
            
            compare_chart_artist = chart_entry[1]['artist'].replace(' ', '').casefold()
            compare_chart_song = chart_entry[1]['song'].replace(' ', '').casefold()
            
            # if the artist and song title are found, save the relevant info
            if compare_chart_artist == compare_track_artist and compare_chart_song == compare_track_title:
                
                chart_entry[1]['artist_uri'] = track_artist_uri
                chart_entry[1]['song_uri']  = track_title_uri

                # continue to the next iteration once a match is found and saved
                break
                    
        ##################################
        # get audio features for all songs
        ##################################
        
        # empty lists to hold desired audio feature scores
        acousticness = []
        duration = []
        danceability = []
        energy = []
        instrumentalness = []
        key = []
        liveness = []
        loudness = []
        mode = []
        speechiness = []
        tempo = []
        time_signature = []
        valence = []
    
        # make the API call for track audio features
        # https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features
    
        song_id = re.search('(?:spotify:track:)(\w+)', chart_entry[1]['song_uri']).group(1)
        chart_entry[1]['song_id'] = song_id
        # API call
        audio = requests.get(base_url + 'audio-features/' + song_id, headers=headers).json()
    
        # save audio features to their respective lists
        chart_entry[1]['acousticness'] = audio['acousticness']
        chart_entry[1]['duration'] = audio['duration_ms']
        chart_entry[1]['danceability'] = audio['danceability']
        chart_entry[1]['energy'] = audio['energy']
        chart_entry[1]['instrumentalness'] = audio['instrumentalness']
        chart_entry[1]['key'] = audio['key']
        chart_entry[1]['liveness'] = audio['liveness']
        chart_entry[1]['loudness'] = audio['loudness']
        chart_entry[1]['mode'] = audio['mode']
        chart_entry[1]['speechiness'] = audio['speechiness']
        chart_entry[1]['tempo'] = audio['tempo']
        chart_entry[1]['time_signature'] = audio['time_signature']
        chart_entry[1]['valence'] = audio['valence']
                
                
        ########################
        # get artist genre data
        ########################
        
        # get artist IDs to use in search
        artist_id = re.search('(?:spotify:artist:)(\w+)', chart_entry[1]['artist_uri']).group(1)
        chart_entry[1]['artist_id'] = artist_id
        artist_ids.append(artist_id)

        # save artist-song combinations to use in dataframe construction
        chart_entry[1]['combo'] = artist_id + song_id
        
        # get the artist's genre(s)
        # Spotify 'artist' search
        # https://developer.spotify.com/documentation/web-api/reference/#/operations/get-an-artist
        
#        try:
        # API call
        get_artist = requests.get(base_url + 'artists/' + artist_id, headers=headers).json()
        artist_genres = get_artist['genres']
        # format genre strings
        artist_genres = [x.replace(' ', '_').lower() for x in artist_genres]
        chart_entry[1]['artist_genres'] = artist_genres
#        print(artist_genres)

#        # for artists with 'NA' URIs
#        except:
#            artist_genres = ['no_genre_data']        
        
    
        # empty dictionary to hold all genres associated with each artist
        artist_genre_dict = {}
        artist_genre_dict[chart_entry[1]['artist_id']] = artist_genres
        all_artist_genres.append(artist_genre_dict)
        
        # save new genres to the list of all genres
        for genre in artist_genres:
            if genre not in all_genres:
                all_genres.append(genre)
    
    
###############
# WORKS TO HERE
###############       

    #        # log progress
    #        clear_output()
    #        checkpoint = timeit.default_timer()
    #        print(f'Found genres for {(a+1)}/{len(artist_ids)} artists ({(a+1)/len(artist_ids)*100:.2f}% complete)')
    #        print(f'{(checkpoint-start)/60:.2f} minutes elapsed')
    #
    #    print('Saving data...')

    all_genres_dict = {}

    for genre in all_genres:
        all_genres_dict[genre] = [0] * len(dfin)
    
    # create a dataframe chart data
    chart_df = pd.DataFrame.from_dict(top40_dict, orient='index')

    # create a dataframe with genre data, sort df columns
    genre_df = pd.DataFrame(all_genres_dict)
    genre_df.columns = sorted(genre_df.columns.tolist())
    genre_df['combo'] = chart_df['combo']
    genre_df.set_index('combo', inplace=True)

    # assign 1's to each artist's genre                
    for column in genre_df:
        
        for row, combo in enumerate(genre_df.index):
            
            for genre in chart_df['artist_genres'][row]:
                                    
                if genre == column:
                    genre_df.loc[combo, column] = 1

                
#    #######################################
#    # drop uncommon genres from the dataset
#    #######################################
#
#    # get the sum of each genre column (exclude the last column)
#    genre_counts = genre_df.sum(axis=0)
#    genre_counts_df = pd.DataFrame(genre_counts, columns=['count'])
#
#    # get genres to drop from the dataset
#    # keeping only the top 50 most common genres
#    drop_genres = genre_counts_df.sort_values(['count'],ascending=False)[50:].index
#
#    # remove low-count genres from the dataset
#    genres_df = full_df.drop(columns=drop_genres)


    ################################
    # output all data to a dataframe
    ################################

    # make a copy of the input dataframe
    trimmed_df = chart_df.copy()
    
    # drop unwanted columns
    trimmed_df.drop(columns = ['id_list', 'hit_list', 'artist_genres', 'artist_uri', 'song_uri'], inplace=True)
    
    # reorder columns for aesthetics
    trimmed_df = trimmed_df[['week', 'position', 'artist', 'song', 'artist_id', 'song_id', 'combo',
                           'acousticness', 'danceability', 'duration', 'energy', 'instrumentalness',
                           'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
                           'time_signature', 'valence']]
    
    # merge all data into a final dataframe
    output_df = trimmed_df.merge(genre_df, how="inner", on="combo")
#
#    # log progress
#    checkpoint = timeit.default_timer()
#    print('Data saved')
#    print(f'Total runtime: {(checkpoint-start)/60:.2f} minutes')
#    print('Done!')
#    #print(type(output_df))
#    #print(type(genre_df))
#    #print(type(full_df))

    # return the dataframe
    return output_df


In [204]:
try40 = top40_dict(top40_df.head(5))

In [205]:
try40

,week,position,artist,song,artist_id,song_id,combo,acousticness,danceability,duration,...,ectofolk,europop,folk,folk_rock,girl_group,lilith,mellow_gold,new_wave_pop,pop_rock,singer-songwriter
0,1997-07-07,1,Hanson,Mmm Bop,0SdiiPkr02EUdekHZJkt58,0lnxrQAd9ZxbhBBe7d8FO8,0SdiiPkr02EUdekHZJkt580lnxrQAd9ZxbhBBe7d8FO8,0.00481,0.683,268653,...,0,0,0,0,0,0,0,0,1,0
1,1997-07-07,2,Meredith Brooks,Bitch,2QmLFuIDtNDmmJY3OtvinN,3i6qNxyVgIdUZTTi5m25EM,2QmLFuIDtNDmmJY3OtvinN3i6qNxyVgIdUZTTi5m25EM,0.01120,0.617,252760,...,0,0,0,0,0,1,0,0,0,0
2,1997-07-07,3,Spice Girls,Say You'll Be There,0uq5PttqEjj3IH1bzwcrXF,1yTQ39my3MoNROlFw3RDNy,0uq5PttqEjj3IH1bzwcrXF1yTQ39my3MoNROlFw3RDNy,0.01490,0.726,235973,...,0,1,0,0,1,0,0,0,0,0
3,1997-07-07,4,Mark Morrison,Return Of The Mack,6V3F8MZrOKdT9fU686ybE9,3jDdpx9PMlfMBS5tOBHFm9,6V3F8MZrOKdT9fU686ybE93jDdpx9PMlfMBS5tOBHFm9,0.00631,0.715,213093,...,0,0,0,0,0,0,0,0,0,0
4,1997-07-07,5,Shawn Colvin,Sunny Came Home,0K7VN4aHxHcEb7PqkfoIVA,4mOxpj82q6n3EO7HBZCelX,0K7VN4aHxHcEb7PqkfoIVA4mOxpj82q6n3EO7HBZCelX,0.34200,0.558,264200,...,1,0,1,1,0,1,1,1,0,1


In [130]:
print(try40.iteritems())

<generator object DataFrame.iteritems at 0x11bcc7430>


In [109]:
index=0
col='song'
try40.loc[index,col] = 'Mmm Bop'

In [867]:
sg_search = f'search?q=artist:Spice%20Girls&type=artist&limit=5'

sg_search_results = requests.get(base_url + sg_search, headers=headers).json()
sg_search_results

{'artists': {'href': 'https://api.spotify.com/v1/search?query=artist%3ASpice+Girls&type=artist&offset=0&limit=5',
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0uq5PttqEjj3IH1bzwcrXF'},
    'followers': {'href': None, 'total': 2886849},
    'genres': ['dance pop', 'europop', 'girl group'],
    'href': 'https://api.spotify.com/v1/artists/0uq5PttqEjj3IH1bzwcrXF',
    'id': '0uq5PttqEjj3IH1bzwcrXF',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab6761610000e5ebcf4a1dced6258416eb219e36',
      'width': 640},
     {'height': 320,
      'url': 'https://i.scdn.co/image/ab67616100005174cf4a1dced6258416eb219e36',
      'width': 320},
     {'height': 160,
      'url': 'https://i.scdn.co/image/ab6761610000f178cf4a1dced6258416eb219e36',
      'width': 160}],
    'name': 'Spice Girls',
    'popularity': 68,
    'type': 'artist',
    'uri': 'spotify:artist:0uq5PttqEjj3IH1bzwcrXF'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/4

In [868]:
sg_search_results['artists']

{'href': 'https://api.spotify.com/v1/search?query=artist%3ASpice+Girls&type=artist&offset=0&limit=5',
 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0uq5PttqEjj3IH1bzwcrXF'},
   'followers': {'href': None, 'total': 2886849},
   'genres': ['dance pop', 'europop', 'girl group'],
   'href': 'https://api.spotify.com/v1/artists/0uq5PttqEjj3IH1bzwcrXF',
   'id': '0uq5PttqEjj3IH1bzwcrXF',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab6761610000e5ebcf4a1dced6258416eb219e36',
     'width': 640},
    {'height': 320,
     'url': 'https://i.scdn.co/image/ab67616100005174cf4a1dced6258416eb219e36',
     'width': 320},
    {'height': 160,
     'url': 'https://i.scdn.co/image/ab6761610000f178cf4a1dced6258416eb219e36',
     'width': 160}],
   'name': 'Spice Girls',
   'popularity': 68,
   'type': 'artist',
   'uri': 'spotify:artist:0uq5PttqEjj3IH1bzwcrXF'},
  {'external_urls': {'spotify': 'https://open.spotify.com/artist/4TL3tMJmDQnnaIdcrrAcmT'},
   'fo

In [982]:
h_hits = requests.get(base_url + 'artists/' + '0SdiiPkr02EUdekHZJkt58' + '/top-tracks?market=US', 
                                       headers=headers).json()
len(h_hits['tracks'])

10

In [1024]:
h_name = 'Hanson'
h_song = f'MMMBop'
no_track_info=[]

for track in h_hits['tracks']:
    
    # if the song is found, save the song and artist IDs
    print('tried')
    if re.search(h_song, track['name']): 
        artist_id = re.search('(?:spotify:artist:)(\w+)', str(track['artists'][0]['uri'])).group(1)
        song_id = re.search('(?:spotify:track:)(\w+)', str(track['uri'])).group(1)
        print(artist_id, song_id)
        # stop looking once the song is found and saved
        break 

#    try: # if the song is found
#        if re.search(sg_song, track[0]['name']):
#            # save the song and artist ID
#            artist_id = re.search('(?:spotify:artist:)(\w+)', track['artists'][0]['uri']).group(1)
#            song_id = re.search('(?:spotify:track:)(\w+)', track['uri']).group(1)
#            print('good')
#            print(artist_id, song_id)
#            break
#
#    except: # some entries don't have track info - skip them and log the track
#        no_track_info.append([sg_name, sg_song])
#        print('not populated')
#        pass


tried
0SdiiPkr02EUdekHZJkt58 0lnxrQAd9ZxbhBBe7d8FO8


In [1087]:
h_hits['tracks']

[{'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0SdiiPkr02EUdekHZJkt58'},
     'href': 'https://api.spotify.com/v1/artists/0SdiiPkr02EUdekHZJkt58',
     'id': '0SdiiPkr02EUdekHZJkt58',
     'name': 'Hanson',
     'type': 'artist',
     'uri': 'spotify:artist:0SdiiPkr02EUdekHZJkt58'}],
   'external_urls': {'spotify': 'https://open.spotify.com/album/3StpQT9Qd87FSeWeQAdg1h'},
   'href': 'https://api.spotify.com/v1/albums/3StpQT9Qd87FSeWeQAdg1h',
   'id': '3StpQT9Qd87FSeWeQAdg1h',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab67616d0000b273184227f002623fc19f44551a',
     'width': 640},
    {'height': 300,
     'url': 'https://i.scdn.co/image/ab67616d00001e02184227f002623fc19f44551a',
     'width': 300},
    {'height': 64,
     'url': 'https://i.scdn.co/image/ab67616d00004851184227f002623fc19f44551a',
     'width': 64}],
   'name': 'Middle Of Nowhere',
   'release_date': '1997-01-01',
   'release_date_pr

In [941]:
re.search(sg_song, str(sg_hits['tracks'][3]['name']))

<re.Match object; span=(0, 19), match="Say You'll Be There">

In [871]:
re.search('(?:spotify:artist:)(\+w)',str(sg_hits['tracks'][0]['artists'][0]['uri']))

In [873]:
re.search('(?:spotify:artist:)(\w+)', sg_hits['tracks'][0]['artists'][0]['uri']).group(1)


'0uq5PttqEjj3IH1bzwcrXF'

In [1028]:
sg_hits[0]['tracks']#[0]['uri']

KeyError: 0